# Libraries

In [2]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 26.4 MB/s eta 0:00:00


In [37]:
import torch
import torch.utils
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertTokenizerFast, BertForSequenceClassification
from torch.optim import Adam


# Prepare dataset

In [6]:
train_df_raw = pd.read_csv("/content/drive/MyDrive/quora data/train.csv")
train_df = train_df_raw.dropna()

In [13]:
print(train_df.head())

      id  qid1  qid2                                          question1  \
29    29    59    60             How should I prepare for CA final law?   
535  535  1068  1069  If you have me, you want to share me. If you s...   
695  695  1387  1388             Do I need to get a tetanus shot again?   
557  557  1112  1113  How many Mechanical Engineers graduate from In...   
836  836  1667  1668   What is difference between net and gross income?   

                                             question2  is_duplicate  
29   How one should know that he/she completely pre...             1  
535  If you have me, you want to share me. If you s...             1  
695                   Do I need to get a tetanus shot?             0  
557  How many mechanical engineers graduate per yea...             1  
836  What is the difference between gross income ve...             1  


In [14]:
train_df, test_df = train_test_split(train_df.head(10000), test_size=0.2, random_state=42)

In [15]:
train_sentences1 = train_df['question1'].tolist()
train_sentences2 = train_df['question2'].tolist()
train_labels = train_df['is_duplicate'].tolist()

test_sentences1 = test_df['question1'].tolist()
test_sentences2 = test_df['question2'].tolist()
test_labels = test_df['is_duplicate'].tolist()

In [38]:
"""
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(train_sentences1, train_sentences2, padding=True, truncation=True, return_tensors='pt')
test_encodings = tokenizer(test_sentences1, test_sentences2, padding=True, truncation=True, return_tensors='pt')
"""

"\ntokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')\n\ntrain_encodings = tokenizer(train_sentences1, train_sentences2, padding=True, truncation=True, return_tensors='pt')\ntest_encodings = tokenizer(test_sentences1, test_sentences2, padding=True, truncation=True, return_tensors='pt')\n"

In [41]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(train_sentences1, train_sentences2, truncation=True, padding=True)
test_encodings = tokenizer(test_sentences1, test_sentences2, truncation=True, padding=True)

In [17]:
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

In [42]:
train_dataset = CustomDataset(train_encodings, train_labels)
test_dataset = CustomDataset(test_encodings, test_labels)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Model

In [19]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
optimizer = Adam(model.parameters(), lr=1e-5)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

# Train, evaluate

In [20]:
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

In [21]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        _, predicted = torch.max(outputs.logits, dim=1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Accuracy: {accuracy}')

Accuracy: 0.71875
